In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.optim as optim

In [10]:
test_dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2022/Digit-Recognizer/test.csv")

In [11]:
test_datas = test_dataset.loc[:, test_dataset.columns != "label"].values / 255

In [12]:
test_x = torch.from_numpy(test_datas).type(torch.float32)

In [13]:
test = torch.utils.data.TensorDataset(test_x)

In [19]:
test_loader = DataLoader(test, shuffle=False)

In [15]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1) # 28
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) # 14

        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1) # 14
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2) # 7

        self.cnn3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1) # 14
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2) # 3

        self.fc1 = nn.Linear(64 * 3 * 3, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)

        out = self.cnn3(out)
        out = self.relu3(out)
        out = self.maxpool3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [16]:
model = CNNModel()

if torch.cuda.is_available():
  model = model.cuda()

In [17]:
modelpath = "/content/drive/MyDrive/Colab Notebooks/2022/Digit-Recognizer/model"

model.load_state_dict(torch.load(f"{modelpath}/model_state_dict.pt"))

<All keys matched successfully>

In [18]:
model.eval()

CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=10, bias=True)
)

In [21]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
outputs = []
for images in test_loader:
    if torch.cuda.is_available():
        images = images[0].cuda()
    output = model(images.view(1, 1, 28, 28))
    predicted = torch.max(output.data, 1)[1]
    outputs.append(predicted)

In [26]:
outputs

[tensor([2], device='cuda:0'),
 tensor([0], device='cuda:0'),
 tensor([9], device='cuda:0'),
 tensor([9], device='cuda:0'),
 tensor([3], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([0], device='cuda:0'),
 tensor([3], device='cuda:0'),
 tensor([0], device='cuda:0'),
 tensor([3], device='cuda:0'),
 tensor([5], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([4], device='cuda:0'),
 tensor([0], device='cuda:0'),
 tensor([4], device='cuda:0'),
 tensor([3], device='cuda:0'),
 tensor([3], device='cuda:0'),
 tensor([1], device='cuda:0'),
 tensor([9], device='cuda:0'),
 tensor([0], device='cuda:0'),
 tensor([9], device='cuda:0'),
 tensor([1], device='cuda:0'),
 tensor([1], device='cuda:0'),
 tensor([5], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([4], device='cuda:0'),
 tensor([2], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([4], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([7], device='cuda:0'),
 tensor([5], device='cuda:0'),
 tensor(

In [29]:
outputs[0].item()

2

In [31]:
from openpyxl import Workbook
write_wb = Workbook()
write_ws = write_wb.active
write_ws['A1'] = 'Predicted'
for output in outputs:
    write_ws.append([output.item()])
write_wb.save('/content/drive/MyDrive/Colab Notebooks/2022/Digit-Recognizer/predicted.xlsx')